In [13]:
from qoop.core import state, ansatz, metric
import qiskit
from qiskit import transpile
from qiskit.quantum_info import Operator, DensityMatrix, Kraus
from scipy.linalg import qr
import numpy as np
import tensorflow as tf

In [14]:
def create_kraus_operators(unitary_matrix):
    '''
        Create a set of Kraus Operators from the input unitary matrix, using QR decomposition
    '''
    kraus_ops = []
    Q, R = qr(unitary_matrix)

    #Q: a 2^N x 2^N matrix, N is the number of qubits
    for q in Q:
        q = np.expand_dims(q, 1)
        kraus_ops.append(q @ np.transpose(np.conjugate(q)))
    return tf.convert_to_tensor(kraus_ops)

def create_unitary_matrix(num_qubits: int):
    """
    Generate a random unitary matrix of size 2^n x 2^n.
    """
    dimension = 2 ** num_qubits
    # Generate a random complex matrix
    random_matrix = np.random.normal(size=(dimension, dimension)) + 0j
    # Perform QR decomposition
    q, _ = qr(random_matrix)
    return q

def calculate_rho2_dephasing(input_rho, num_qubits: int, gamma: float): #for verification
    '''
    Calculate rho2 by dephasing
    '''
    
    # Convert DensityMatrix to numpy array
    rho = input_rho.data
    # Define the Pauli-Z matrix
    sigma_z = np.array([[1, 0], [0, -1]])

    # Calculate the factors
    alpha = 1 + np.sqrt(1 - gamma)
    beta = 1 - np.sqrt(1 - gamma)
    
    # n qubits => qubit thứ n => I @ I @ .... sigma_z (n) @....I
    # Loop for multiple qubits
    for i in range(num_qubits):
        # Create the tensor product of Pauli-Z matrices for all qubits
        sigma_z_i = np.eye(1)
        for j in range(num_qubits):
            if j == i:
                sigma_z_i = np.kron(sigma_z_i, sigma_z)
            else:
                sigma_z_i = np.kron(sigma_z_i, np.eye(2))
        
        # Apply the dephasing formula
        rho = alpha * rho + beta * (sigma_z_i @ rho @ sigma_z_i)
        # Normalize the density matrix (optional, depending on the context)
        rho /= np.trace(rho)

    return rho

def calculate_rho2_from_unitary(rho, unitary_matrix):
    '''
    Calculate rho' by applying U @ rho @ U(dagger)
    '''
    rho_2 = unitary_matrix  @ rho.data @ np.transpose(np.conjugate(unitary_matrix))

    return rho_2

def calculate_rho2_from_kraus_operators(rho, kraus_operators):
    '''
    Calculate rho' by applying K @ rho @ K(dagger)
    '''
    rho_2 = sum(K @ rho @ np.transpose(np.conjugate(K)) for K in kraus_operators)

    return rho_2

def calculate_rho_3 (rho_2, kraus_operators):
    '''
        sum(K @ rho_2 @ K(dagger)) //rho" = rho
    '''

    rho_3 = sum(K @ rho_2 @ np.transpose(np.conjugate(K)) for K in kraus_operators)
    return rho_3





In [15]:
def compilation_trace_fidelity(rho, sigma):
    """Calculating the fidelity metric

    Args:
        - rho (DensityMatrix): first density matrix
        - sigma (DensityMatrix): second density matrix

    Returns:
        - float: trace metric has value from 0 to 1
    """
    rho_2 = tf.linalg.sqrtm((tf.linalg.sqrtm(rho)) @ (rho))

    # Cast to a supported type
    real_part = tf.math.real(rho_2)
    imaginary_part = tf.math.imag(rho_2)

    # Check for NaNs in both real and imaginary parts
    contains_nan_real = tf.reduce_any(tf.math.is_nan(real_part))
    contains_nan_imag = tf.reduce_any(tf.math.is_nan(imaginary_part))

    contains_nan = contains_nan_real or contains_nan_imag

    if contains_nan == True:
        rho_2 = rho

    return tf.linalg.trace(
            rho_2
            @ (tf.linalg.sqrtm(sigma))
        )
def frobenius_norm(rho, sigma):
    """
    Compute the Frobenius norm between two matrices.

    Parameters:
    rho (numpy.ndarray): The first matrix.
    sigma (numpy.ndarray): The second matrix.

    Returns:
    float: The Frobenius norm between rho and rho_prime.
    """
    
    # Ensure the matrices are TensorFlow tensors
    rho = tf.convert_to_tensor(rho, dtype=tf.complex128)
    sigma = tf.convert_to_tensor(sigma, dtype=tf.complex128)

    # Compute the difference between the matrices
    diff = rho - sigma

    # Compute the Frobenius norm
    #norm = tf.linalg.normalize(diff, ord='fro')
    norm = tf.sqrt(tf.reduce_sum(tf.square(diff)))
    return norm

#cost func to compare 2 given rhos
def cost(rho, rho_3):
    return tf.square(frobenius_norm(rho, rho_3))
    #return 1 - compilation_trace_fidelity(rho, rho_3)

In [16]:
#Auto Diff
'''
    rho: The initial rho
    unitary_matrix: randomly generated circle in initialization step
    kraus_operators: set of kraus operators
    n: number of qubits
    alpha: learning rate (?)'''
def calculate_derivative(rho, rho2, kraus_operators, num_qubits, alpha=0.1):
    tensorKraus = tf.Variable(kraus_operators)
    with tf.GradientTape() as tape:
        rho3 = calculate_rho_3(rho2, tensorKraus)
        f = cost(rho, rho3) 
    
    # Calculate the gradient
    c = tape.gradient(f, tensorKraus)
    
    # Ensure proper reshaping (ensure this matches the dimensions of your system)
    c = tf.reshape(c, (2**num_qubits, 2**num_qubits, 2**num_qubits))

    # Calculate projection
    proj = c - tensorKraus @ (np.transpose(np.conjugate(c)) @ tensorKraus + np.transpose(np.conjugate(tensorKraus)) @ c) / 2

    # Update the Kraus operators
    updated_kraus_operators = tensorKraus - alpha * proj
    
    # Return the updated Kraus operators
    return updated_kraus_operators



In [17]:
# Adam optimizer function for updating Kraus operators
def calculate_derivative_adam(rho, rho2, kraus_operators, m, v, num_qubits, t, alpha=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
    tensorKraus = tf.Variable(kraus_operators, dtype=tf.complex128)

    beta1 = tf.constant(beta1, dtype=tf.complex128)
    beta2 = tf.constant(beta2, dtype=tf.complex128)
    t = tf.constant(t, dtype=tf.complex128)

    with tf.GradientTape() as tape:
        rho3 = calculate_rho_3(rho2, tensorKraus)
        f = cost(rho, rho3)
    
    # Calculate the gradient
    c = tape.gradient(f, tensorKraus)
    
    # Ensure proper reshaping (ensure this matches the dimensions of your system)
    c = tf.reshape(c, (2**num_qubits, 2**num_qubits, 2**num_qubits))

    # Calculate projection
    proj = c - tensorKraus @ (np.transpose(np.conjugate(c)) @ tensorKraus + np.transpose(np.conjugate(tensorKraus)) @ c) / 2

    # Update Adam variables
    m = beta1 * m + (1 - beta1) * proj
    v = beta2 * v + (1 - beta2) * tf.math.square(proj)

    # Bias correction
    m_hat = m / (1 - tf.pow(beta1, t + 1))
    v_hat = v / (1 - tf.pow(beta2, t + 1))

    # Update the Kraus operators using Adam update rule
    updated_kraus_operators = tensorKraus - alpha * m_hat / (tf.math.sqrt(v_hat) + epsilon)

    return updated_kraus_operators, m, v

In [18]:
def optimize(rho, rho2, kraus_operators, num_qubits, alpha=0.001, num_loop = 1000):
    kraus_operators_copy = tf.identity(kraus_operators)
    # try looping manually
    for i in range (0, num_loop):
        rho3 = calculate_rho_3(rho2, kraus_operators_copy)
        _cost = cost(rho, rho3) 

        #Update Kraus Operators
        kraus_operators_copy=calculate_derivative(rho, rho2, kraus_operators_copy, num_qubits, alpha)

        #Reshape
        kraus_operators_copy = tf.reshape(kraus_operators_copy, (2**num_qubits,2**num_qubits,2**num_qubits))

        print('Loop No.' + str(i) + ': ' + (str)(_cost.numpy()))
        #if (_cost.numpy() < 1e-4): 
         #   break
    return kraus_operators_copy

In [19]:
def optimize_adam(rho, rho2, kraus_operators, num_qubits, alpha=0.1, num_loop = 1000):
    kraus_operators_copy = tf.identity(kraus_operators)
    # Initialize m, v to zero matrices
    m = tf.zeros_like(kraus_operators_copy, dtype=tf.complex128)
    v = tf.zeros_like(kraus_operators_copy, dtype=tf.complex128)
    # try looping manually
    for i in range (0, num_loop):
        rho3 = calculate_rho_3(rho2, kraus_operators_copy)
        _cost = cost(rho, rho3) 

        #Update Kraus Operators
        kraus_operators_copy, m, v =calculate_derivative_adam(rho, rho2, kraus_operators_copy, m, v, num_qubits, i, alpha)

        #Reshape
        kraus_operators_copy = tf.reshape(kraus_operators_copy, (2**num_qubits,2**num_qubits,2**num_qubits))
        m = tf.reshape(m, (2**num_qubits,2**num_qubits,2**num_qubits))
        v = tf.reshape(v, (2**num_qubits,2**num_qubits,2**num_qubits))
        #print('K(t)K\n',sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry)) # I

        print('Loop No.' + str(i) + ': ' + (str)(_cost.numpy()))
        #if (_cost.numpy() < 1e-4): 
         #   break
    return kraus_operators_copy

In [21]:
def V(num_qubits: int):
    """
        Create a ansatz V with n qubits and assign its parameters
        Args:
        - num_qubits (int): number of qubits

        Returns:
        - qiskit.QuantumCircuit: parameter assigned Quantum circuit
    """
    #Assign random parameter
    circuit = ansatz.graph(num_qubits=num_qubits)
    num_params = circuit.num_parameters
    x0 = 2 * np.pi * np.random.random(num_params)
    circuit = circuit.assign_parameters(dict(zip(circuit.parameters, x0)))
    return circuit

In [22]:
def quantum_process_tomography_dephasing (num_qubits, gamma, num_loop, rho, kraus_operators):
    rho2 = calculate_rho2_dephasing(rho, num_qubits, gamma)
    # Optimizing
    out_kraus_operators = optimize_adam(rho, rho2, kraus_operators, num_qubits, 0.005, num_loop)

    _cost = 1 - compilation_trace_fidelity(rho, calculate_rho_3(rho2, out_kraus_operators))
    return rho2, out_kraus_operators, _cost


In [23]:

def init (num_qubits):
    """Initialize a |+⟩^{⊗n} state as rho and a random first set of Kraus operators:
        Args:
        - num_qubits (int): number of qubits

        Returns:
        - DensityMatrix: |+⟩^{⊗n} state
        - tf.Tensor: set of Kraus operators (shape: 2^n * 2^n * 2^n)
    """
    # Create |+⟩^{⊗n} state
    plus_state = (1/np.sqrt(2)) * np.array([1, 1])
    initial_state_vector = plus_state
    for _ in range(num_qubits - 1):
        initial_state_vector = np.kron(initial_state_vector, plus_state)
    rho_matrix = np.outer(initial_state_vector, initial_state_vector.conj())
    rho = DensityMatrix(rho_matrix)

    # Generate a random unitary_matrix and random set of kraus_operators
    unitary_matrix = create_unitary_matrix(num_qubits=num_qubits)

    # Get first set of kraus_operators
    kraus_operators = create_kraus_operators(unitary_matrix=unitary_matrix) 
    return rho, kraus_operators

In [ ]:
'''plotting
fix ite, run gamma & num-qubit
fix gamma, fix num-qubit, tính p' và p'2 dagger, 
lưu kraus và cost cuối cùng
'''
import os
experiment_folder = 'experiment_result'
g_shift = 0.025

for num_qubits in range(1, 6):
    g = 0
    rho, kraus_operators = init(num_qubits)
    while g < 1:
        folder_path = str(num_qubits)+"_qubits_"+"{:.3f}".format(g)
        folder_path = os.path.join(experiment_folder, folder_path)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        rho2, out_kraus_operators, _cost = quantum_process_tomography_dephasing(num_qubits, g, 700, rho, kraus_operators)
        rho2_file_path = os.path.join(folder_path, 'rho2.txt')
        kraus_file_path = os.path.join(folder_path, 'kraus_operators.txt')
        cost_file_path = os.path.join(folder_path, 'cost.txt')
        
        
        g = round(g + g_shift, 3)
        with open(rho2_file_path, 'w') as file:
            file.write(str(rho2))
        with open(kraus_file_path, 'w') as file:
            file.write(str(out_kraus_operators))
        with open(cost_file_path, 'w') as file:
            file.write(str(_cost))

In [33]:
rho, kraus_operators = init(3)
rho2, out_kraus_operators, _cost = quantum_process_tomography_dephasing(3, g, 700, rho, kraus_operators)

Loop No.0: (0.6993273709433542+0j)
Loop No.1: (0.5709002244879213+0j)
Loop No.2: (0.43715718597750264+0j)
Loop No.3: (0.3074625552991814+0j)
Loop No.4: (0.1958802630583979+0j)
Loop No.5: (0.1182029167350157+0j)
Loop No.6: (0.08638473347004992+0j)
Loop No.7: (0.09628338452438663+0j)
Loop No.8: (0.11860170058001268+0j)
Loop No.9: (0.12393123317307266+0j)
Loop No.10: (0.10673772766007086+0j)
Loop No.11: (0.07714030447532319+0j)
Loop No.12: (0.047669763636059716+0j)
Loop No.13: (0.026660952794958375+0j)
Loop No.14: (0.016679294437021982+0j)
Loop No.15: (0.015788711792264216+0j)
Loop No.16: (0.019945294558193577+0j)
Loop No.17: (0.025117382037137124+0j)
Loop No.18: (0.028464980800081807+0j)
Loop No.19: (0.02865030543026221+0j)
Loop No.20: (0.025650462218078932+0j)
Loop No.21: (0.02038125798490963+0j)
Loop No.22: (0.014277229617733709+0j)
Loop No.23: (0.00886278460360051+0j)
Loop No.24: (0.0053295670244059914+0j)
Loop No.25: (0.0041841021171512924+0j)
Loop No.26: (0.005091473195784312+0j)
Lo

In [34]:
rho2_kraus = calculate_rho2_from_kraus_operators(rho, out_kraus_operators)

In [35]:
compilation_trace_fidelity(rho2, np.transpose(np.conjugate(rho2_kraus)))

<tf.Tensor: shape=(), dtype=complex128, numpy=(0.9999986978752062+0j)>

In [ ]:
'''rho |0>
U (rho) -> rho2
Tạo một tập các toán tử Kraus ban đầu //random 
=> tạo unitary_matrix random, QR decomposition
kraus_operators(rho2) -> rho3,
optimize rho3 ~ rho
rho - U ---- U(dagger) - rho
        rho2 kraus_operators -> rho'''

'rho |0>\nU (rho) -> rho2\nTạo một tập các toán tử Kraus ban đầu //random \n=> tạo unitary_matrix random, QR decomposition\nkraus_operators(rho2) -> rho3,\noptimize rho3 ~ rho\nrho - U ---- U(dagger) - rho\n        rho2 kraus_operators -> rho'